In [ ]:
from pathlib import Path

from src.config import DOCUMENTS_DIR, TEST_DIR
from src.llm_query import call_llm
from src.retriever import jaccard_retriever, get_fittest_chunk, tfidf_retriever
from src.chunk import trunkate_on_h2, find_chunk_indices_with_title
import requests
import json


path = Path(DOCUMENTS_DIR)


In [ ]:
texts = []
for filename in path.glob("*.md"):
    with open(filename) as f:
        texts.append(f.read())


## trucate file into chunks

In [ ]:
chunks = trunkate_on_h2(texts)

In [ ]:
chunks

In [ ]:
chunk_tensors_id = find_chunk_indices_with_title(chunks, "burn::tensor# Struct PrintOptions")

In [ ]:
tensor_chunk_list = [chunks[i] for i in chunk_tensors_id]

In [ ]:
tensor_chunk_list

In [ ]:
with open(TEST_DIR / 'questions_evaluation.json', 'r') as f:
    eval_json = json.load(f)
eval_json

In [ ]:
fittest_doc_id = tfidf_retriever(chunks, eval_json[0]['question'])[0]
fittest_doc = chunks[fittest_doc_id]

In [ ]:
fittest_doc

In [ ]:
response = call_llm(eval_json[0]['question'], fittest_doc,llm="llama2")

In [ ]:
response

In [ ]:
from src.evaluation import evaluate_rag

In [ ]:
evaluate_rag(eval_json, tfidf_retriever, call_llm, chunks)